In [1]:
import os
from nltk.stem import PorterStemmer
import math


allfiles_tokens = [] # a list of doc of tokens
vocabulary = dict() # all unrepaeat tokens
sorted_vocabulary = [] 


# tokenize

path = './IRTM/'
filenames = os.listdir(path)
for file in filenames:
    fp = open(f'{path}{file}', 'r')    
    singlefile_tokens = []
    for line in fp:
        modified_line = []
        for i in line:
            if i.isalpha() or i == ' ':
                modified_line.append(i)
        modified_line = ''.join(modified_line)

        singleline_tokens = modified_line.split(' ')
        for token in singleline_tokens:
            if token:
                singlefile_tokens.append(token)
    fp.close()

    for i in range(len(singlefile_tokens)):
        singlefile_tokens[i] = singlefile_tokens[i].lower()

    ps = PorterStemmer()
    for i in range(len(singlefile_tokens)):
        singlefile_tokens[i] = ps.stem(singlefile_tokens[i])

    singlefile_results = []
    nltk_stopwords = ['news', 'im', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    nltk_stopwords = set(nltk_stopwords)
    for i in range(len(singlefile_tokens)):
        if singlefile_tokens[i] not in nltk_stopwords:
            singlefile_results.append(singlefile_tokens[i])
    
    allfiles_tokens.append(singlefile_results)

    
# save in dictionary

allfiles_tokens_set = [None] * len(allfiles_tokens) # a list of doc of unrepeat tokens

for i, document in enumerate(allfiles_tokens):
    allfiles_tokens_set[i] = set(document)
    for token in allfiles_tokens_set[i]:
        if token in vocabulary:
            vocabulary[token] += 1
        else:
            vocabulary[token] = 1



# sort the dictionary

for token in vocabulary:
    sorted_vocabulary.append(token)
sorted_vocabulary.sort()



In [10]:
number_of_class = 13
training_set = [None] * number_of_class
count_docs = 0

fp = open('training.txt', 'r')
for line in fp:
    words = line.strip().split(' ')
    class_name = int(words[0])
    training_set[class_name-1] = []
    for i in range(1, len(words)):
        training_set[class_name-1].append(words[i])
        count_docs += 1
fp.close()


In [172]:
from collections import Counter
import heapq

prior_each_class = [0] * number_of_class
count_docs_each_class = [0] * number_of_class
tokens_each_class = [None] * number_of_class
count_tokens_each_class = [0] * number_of_class
count_token_freq = dict()
selected_token = []

# count prior

for i in range(number_of_class):
    for document in training_set[i]:
        count_docs_each_class[i] += 1
for i in range(number_of_class):
    prior_each_class[i] = count_docs_each_class[i] / count_docs

    
# count all tokens in each class

for i in range(number_of_class):
    tokens_each_class[i] = []
    for document in training_set[i]:
        for token in allfiles_tokens[int(document)-1]:
            tokens_each_class[i].append(token)
            

# count numbers of all tokens in each class
            
for i in range(number_of_class):
    count_tokens_each_class[i] = len(tokens_each_class[i])


# count each token frequency in each class
    
for word in vocabulary:
    count_token_freq[word] = []
    for class_name in range(number_of_class):
        count_token_freq[word].append(Counter(tokens_each_class[class_name])[word])

# count Likelihood Ratios in each class
        
for class_c in range(number_of_class):
    likelihood_tokens = dict()
    top_k_tokens = []
    for token in tokens_each_class[class_c]:
        n_11 = count_token_freq[token][class_c] # present and on topic
        n_01 = sum(count_token_freq[token]) - n_11  # present and off topic
        n_10 = count_tokens_each_class[class_c] - n_11 # absent and on topic
        n_00 = sum(count_tokens_each_class) - sum(count_token_freq[token]) - n_10 # absent and off topic
        N = sum(count_tokens_each_class)
        
        a = (((n_11+n_01)/N)**n_11) * ((1-(n_11+n_01)/N)**n_10) * (((n_11+n_01)/N)**n_01) * ((1-(n_11+n_01)/N)**n_00)
        b = ((n_11/(n_11+n_10))**n_11) * ((1-n_11/(n_11+n_10))**n_10) * ((n_01/(n_01+n_00))**n_01) * ((1-n_01/(n_01+n_00))**n_00)
        if a!=0 and b != 0:
            x = -2 * math.log(a/b)
        else:
            x = 0
        likelihood_tokens[token] = x
        
    y = heapq.nlargest(38, likelihood_tokens.items(), key=itemgetter(1))
    top_k_tokens = list(map(itemgetter(0), y))
    
    for token in top_k_tokens:
        selected_token.append(token)
        



In [208]:
selected_vocabulary = set()
selected_tokens_each_class = [None] * number_of_class
count_selected_tokens_each_class = [None] * number_of_class
count_selected_token_freq = dict()
selected_vocabulary = set(selected_token)
condProb = dict()

for i in range(number_of_class):
    selected_tokens_each_class[i] = []
    for token in tokens_each_class[i]:
        if token in selected_vocabulary:
            selected_tokens_each_class[i].append(token)
    count_selected_tokens_each_class[i] = len(selected_tokens_each_class[i])
    
for token in selected_vocabulary:
    count_selected_token_freq[token] = []
    condProb[token] = []
    for i in range(number_of_class):
        count_selected_token_freq[token].append(Counter(selected_tokens_each_class[i])[token]) 
    for i in range(number_of_class):
        a = count_selected_token_freq[token][i] + 1
        b = count_selected_tokens_each_class[i] + len(selected_vocabulary)
        condProb[token].append(a/b)
        

       





In [214]:
total_training = []

for i in range(number_of_class):
    for document in training_set[i]:
        total_training.append(int(document))
        
total_training = sorted(total_training)

savefilename = 'hw3_1.csv'  
rp = open(savefilename, 'w')
rp.write('Id' + ',' + 'Value' + '\n')

for document in range(1, len(allfiles_tokens)+1):
    if document not in total_training:
        test_tokens = allfiles_tokens[document-1]
        score = [0] * number_of_class
        for i in range(number_of_class):
            for token in test_tokens:
                if token in selected_vocabulary:
                    score[i] += math.log(condProb[token][i])
        class_result = int(score.index(max(score))) + 1
        rp.write(f'{document},{class_result}\n')
        
rp.close()
        
    






